In [ ]:
!pip install transformers datasets

In [ ]:
from datasets import load_datasets
raw_datasets = load_dataset("squad")
raw_datasets

In [ ]:
raw_datasets["train"][1]["title"]

In [ ]:
raw_datasets["train"][1]["context"]

In [ ]:
raw_datasets["train"][1]["question"]

In [ ]:
raw_datasets["train"][1]["answers"]